In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configure plot style
plt.style.use('seaborn')
sns.set_palette('husl')


In [ ]:
# Database connection
db_url = f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DB')}"
engine = create_engine(db_url)

# Load data
query = """
SELECT l.*, u.name as user_name
FROM lancamento l
JOIN user u ON l.user_id = u.id
"""
df = pd.read_sql(query, engine)


In [ ]:
# Prepare temporal data
df['month'] = df['data'].dt.to_period('M')
monthly_summary = df.groupby(['month', 'tipo'])['valor'].sum().unstack()

# Plot monthly evolution
plt.figure(figsize=(15, 6))
monthly_summary.plot(kind='bar', stacked=True)
plt.title('Evolução Mensal de Receitas e Despesas')
plt.xlabel('Mês')
plt.ylabel('Valor (R$)')
plt.xticks(rotation=45)
plt.legend(title='Tipo')
plt.tight_layout()
plt.show()


In [ ]:
# Category analysis
category_summary = df.groupby(['categoria', 'tipo'])['valor'].sum().unstack()

# Plot category distribution
fig = px.treemap(df, 
                 path=['tipo', 'categoria'],
                 values='valor',
                 title='Distribuição de Valores por Categoria')
fig.show()


In [ ]:
# Statistical summary
stats_summary = df.groupby('tipo')['valor'].agg(['count', 'mean', 'std', 'min', 'max'])
print("\nResumo Estatístico por Tipo:")
print(stats_summary)

# Box plot
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='tipo', y='valor')
plt.title('Distribuição de Valores por Tipo')
plt.show()


In [ ]:
# Create temporal features
df['day_of_week'] = df['data'].dt.dayofweek
df['day_of_month'] = df['data'].dt.day
df['month_num'] = df['data'].dt.month

# Correlation analysis
numeric_cols = ['valor', 'day_of_week', 'day_of_month', 'month_num']
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlação')
plt.show()
